In [1]:
''' GOMS와 IBSS규정 적용현황을 비교하기 위한 스크립트'''
import pandas as pd
import numpy as np
import GladUtil as gld

### goms규정

In [2]:
infile = 'D:/글로벌백업(180329)/05.GOMS수수료작업/규정_GOMS_2106지급.xlsx'
df_fc = pd.read_excel(infile,sheet_name='fc',usecols='b:d,g:i,o:r,m',dtype=object)
df_fc = df_fc[df_fc.퇴사일자.isnull()]
df_fc['지점'] = df_fc.apply(lambda x: gld.Metro(x['지점'], x['직할지점'], x['팀']), axis=1) #글래드 지점 변환

df_brh = pd.read_excel(infile,sheet_name='brh',usecols='c,f:h,k:r,j',dtype=object)
df_brh = df_brh[df_brh.폐쇄일자.isnull()]
df_brh['지점'] = df_brh.apply(lambda x: gld.Metro(x['지점'], x['직할지점'], x['팀']), axis=1) #글래드 지점 변환
df_brh['지사'] = df_brh.지점.map(lambda x: '지사'  if x in gld.GROUP2 else ('직영' if x in gld.GROUP1 else '오류')) # 지사구분

In [3]:
''' 보종별 glad 기본 규정 find '''
FC_LIFE = df_brh[ (df_brh['지점'].isnull()) & (df_brh.직할지점.isnull()) &  (df_brh.FC생보규정 !='미선택') ]['FC생보규정'].values[0]
FC_LONG = df_brh[ (df_brh['지점'].isnull()) & (df_brh.직할지점.isnull()) &  (df_brh.FC장기규정 !='미선택') ]['FC장기규정'].values[0]
FC_CAR  = df_brh[ (df_brh['지점'].isnull()) & (df_brh.직할지점.isnull()) &  (df_brh.FC자동차규정 !='미선택') ]['FC자동차규정'].values[0]
FC_GEN  = df_brh[ (df_brh['지점'].isnull()) & (df_brh.직할지점.isnull()) &  (df_brh.FC일반규정 !='미선택') ]['FC일반규정'].values[0]
print(FC_LIFE)
print(FC_LONG)
print(FC_CAR)
print(FC_GEN)

''' 지점규정없으면 기본규정을 적용'''
df_brh.loc[df_brh['FC생보규정']=='미선택','FC생보규정'] = FC_LIFE
df_brh.loc[df_brh['FC장기규정']=='미선택','FC장기규정'] = FC_LONG
df_brh.loc[df_brh['FC자동차규정']=='미선택','FC자동차규정'] = FC_CAR
df_brh.loc[df_brh['FC일반규정']=='미선택','FC일반규정'] = FC_GEN
df_brh.drop_duplicates(inplace=True)

''' 지점규정과 join '''
df_brh1 = df_brh[['지급년월','지점','직할지점','팀','FC생보규정','FC장기규정','FC자동차규정','FC일반규정','지사']].drop_duplicates()
df = df_fc.merge(df_brh1, on=['지점','직할지점','팀'],suffixes=('','_brh'))

''' FC규정이 없으면 지점규정을 적용 '''
df['FC생보규정'] = [ y if x == '미선택' else x for (x,y) in zip(df['FC생보규정'], df['FC생보규정_brh']) ]
df['FC장기규정'] = [ y if x == '미선택' else x for (x,y) in zip(df['FC장기규정'], df['FC장기규정_brh']) ]
df['FC자동차규정'] = [ y if x == '미선택' else x for (x,y) in zip(df['FC자동차규정'], df['FC자동차규정_brh']) ]
df['FC일반규정'] = [ y if x == '미선택' else x for (x,y) in zip(df['FC일반규정'], df['FC일반규정_brh']) ]

cols = df.columns[~df.columns.str.contains('brh')]
df = df[cols]
df.to_clipboard()

대_생보_65
대_2021-04 손보수수료(GLD_700)
대_2020-02 자동차수수료(표준FC_11%)
대_201205-글로벌금융공통FC규정


### ibss 규정

In [4]:
infile = 'D:/글로벌백업(180329)/05.GOMS수수료작업/IBSS규정적용.xlsx'
df_ibss = pd.read_excel(infile,skiprows=1,dtype=object)
APLY_YM = '202105'
PAY_YM  = '202106'
cols = '본부 지점 팀 담당 사원번호 생보_적용월 생보_규정 생보_OR 생보_환수율 생보_성과 장기_적용월 장기_규정 장기_OR 장기_환수율 장기_성과 자동차_적용월 자동차_규정 자동차_OR 자동차_성과 일반_적용월 일반_규정'
cols = cols.split(' ')
df_ibss.columns = cols

cols1 = df_ibss.columns[~df_ibss.columns.str.contains('본부|OR|팀|성과')]
df_ibss = df_ibss[cols1]
df_ibss.fillna('',inplace=True)

In [5]:
keys = ['지점','담당','사원번호']
def create_df(df_ibss, 보종명,구분):
    aply_nm1 = 보종명+'_적용월'
    aply_nm2 = 보종명+'_규정'
    df_tmp = df_ibss[df_ibss[aply_nm1] <= APLY_YM]
    df_tmp = df_tmp[df_tmp[aply_nm2] !=''][keys + [aply_nm1,aply_nm2 ]]
    df_tmp= df_tmp.groupby(keys).max().reset_index()
    df_tmp['지급년월'] = PAY_YM
    if 구분=='OR':
        return df_tmp[df_tmp['담당']==''] 
    else:
        return df_tmp

''' 보종별 ibss규정 생성'''
df_life = create_df(df_ibss,'생보','FC')
df_long = create_df(df_ibss,'장기','FC')
df_car  = create_df(df_ibss,'자동차','FC')
df_gen  = create_df(df_ibss,'일반','FC')

### 비교

In [7]:
def create_compare(df, df2, 보종명,구분):
    aply_nm1 = 구분+보종명+'규정'
    aply_nm2 = 보종명+'_규정'
    df_goms = df[['지급년월','지점','사원명','사원번호',aply_nm1]]
    df_comp = df_goms.merge(df2[['사원번호',aply_nm2,'지급년월']],on=['사원번호','지급년월'],indicator=True)
    return df_comp

''' fc규정비교 : goms vs ibss '''
df_comp_life = create_compare(df,df_life,'생보','FC')
df_comp_long = create_compare(df,df_long,'장기','FC')
df_comp_car  = create_compare(df,df_car,'자동차','FC')
df_comp_gen  = create_compare(df,df_gen,'일반','FC')
# df_comp_life.head()

In [10]:
# df_comp_gen.to_clipboard()
outfile = 'D:/글로벌백업(180329)/05.GOMS수수료작업/비교결과_FC.xlsx'
with pd.ExcelWriter(outfile) as writer:
        df_comp_life.to_excel(writer, sheet_name='생보', index=False)
        df_comp_long.to_excel(writer, sheet_name="장기", index =False)
        df_comp_car.to_excel(writer, sheet_name="자동차", index =False)
        df_comp_gen.to_excel(writer, sheet_name="일반", index =False)

In [8]:
''' 지사규정 비교 '''
# goms 지사
df_tmp = df_brh.replace('미선택',np.nan)[['지급년월','지점','OR생보규정','OR장기규정','OR자동차규정','OR일반규정','지사']]
df_tmp.dropna(subset=['OR생보규정','OR장기규정','OR자동차규정','OR일반규정'],how='all',inplace=True)
df_or_goms = df_tmp[ df_tmp['지사'] != '직영'].drop_duplicates(subset=['지급년월','지점']).sort_values('지점')

# ibss지사
df_or_ibss_life = create_df(df_ibss,'생보','OR')
df_or_ibss_long = create_df(df_ibss,'장기','OR')
df_or_ibss_car  = create_df(df_ibss,'자동차','OR')
df_or_ibss_gen  = create_df(df_ibss,'일반','OR')
''' 지사 기본규정 '''
OR_CAR = df_or_goms[ (df_or_goms['지점'].isnull()) &  (df_or_goms.OR자동차규정.notnull()) ] ['OR자동차규정'].values[0]
OR_GEN = df_or_goms[ (df_or_goms['지점'].isnull()) &  (df_or_goms.OR일반규정.notnull()) ] ['OR자동차규정'].values[0]
print(OR_CAR)
print(OR_GEN)
df_or_goms.loc[df_or_goms['OR자동차규정'].isnull(),'OR자동차규정'] = OR_CAR
df_or_goms.loc[df_or_goms['OR일반규정'].isnull(),'OR일반규정'] = OR_GEN

''' ibss규정과 join '''
df_or_goms = df_or_goms.merge(df_or_ibss_life[['생보_규정','지점']],on='지점')
df_or_goms = df_or_goms.merge(df_or_ibss_long[['장기_규정','지점']],on='지점')
df_or_goms = df_or_goms.merge(df_or_ibss_car[['자동차_규정','지점']],on='지점')
df_or_goms = df_or_goms.merge(df_or_ibss_gen[['일반_규정','지점']],on='지점')

대_OR_2020-02 자동차수수료(성과지점)
대_OR_2020-02 자동차수수료(성과지점)


In [9]:
# df_or_goms.to_clipboard()
outfile = 'D:/글로벌백업(180329)/05.GOMS수수료작업/비교결과_OR.xlsx'
with pd.ExcelWriter(outfile) as writer:
        df_or_goms.to_excel(writer, sheet_name='OR', index=False)